In [9]:
#TASK 1

import re
from collections import Counter
file = open('task1text.txt', 'r', encoding='utf-8')
data = file.read()

data = re.sub(r'[^а-яА-ЯёЁ\d\s\.\,]', '', data)

trash = set()
data = data.split()

counter = Counter(data)

for i in counter:
    if counter[i] > 10:
        trash.add(i)

clear = []
for i in data:
    if i not in trash:
        clear.append(i)
data = ' '.join(clear)
print(data)
print('\nИз контекста мне не понятно является ли слово плод очепяткой или так задуманно. На всякий точечно заменю на плот\n')
newdata = data.replace("плод", "плот")
print(newdata)

Тот, кто построил плод на летней школе в 2023 году, участвовал в покушении на господина. Имена преступников я не могу вам сообщить, но у вас есть возможность поехать на летнюю школу чтобы выяснить что там произошло и собрать информацию на месте. Прошу найдите виновных в покушении. Это необходимо для сохранения чести летней школы.

Из контекста мне не понятно является ли слово плод очепяткой или так задуманно. На всякий точечно заменю на плот

Тот, кто построил плот на летней школе в 2023 году, участвовал в покушении на господина. Имена преступников я не могу вам сообщить, но у вас есть возможность поехать на летнюю школу чтобы выяснить что там произошло и собрать информацию на месте. Прошу найдите виновных в покушении. Это необходимо для сохранения чести летней школы.


In [5]:
#TASK 5
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from collections import Counter
df = pd.read_csv("sample.tsv", sep="\t", header=None, names=["label", "message"])
stopwords = pd.read_csv("english.csv", header=None)
stop_words = set(stopwords[0].str.strip().tolist())

df["message"] = df["message"].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))
df["message"] = df["message"].apply(lambda x: word_tokenize(x))
print('\nПриведено все к нижнему регистру, удалены символы не являющеся буквами и пробелами.\n')
print(df.message)

lemmatizer = WordNetLemmatizer()
df["message"] = df["message"].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])
print('\nЛемматизация\n')
print(df.message)


df["message"] = df["message"].apply(lambda tokens: [word for word in tokens if word not in stop_words])
print('\nУдаление стоп слов\n')
print(df.message)



all_ham_words = [word for message in df[df['label'] == "ham"]["message"] for word in message]
all_spam_words = [word for message in df[df['label'] == "spam"]["message"] for word in message]
counterHam = Counter(all_ham_words)
counterSpam = Counter(all_spam_words)
longCounterHam = [(word, count) for word, count in counterHam.items() if len(word) >= 3]
longCounterSpam = [(word, count) for word, count in counterSpam.items() if len(word) >= 3]
print("\nКоличество встречающихся 10 слов длиннее 3х\n")
sortHum = sorted(longCounterHam, key=lambda x: x[1], reverse=True)
sortSpam = sorted(longCounterSpam, key=lambda x: x[1], reverse=True)
print(f'Не спам сообщения{sortHum[:10]}')
print(f'Спам сообщения{sortSpam[:10]}')

all_words = [word for message in df["message"] for word in message]
counter_all = Counter(all_words)

filtered_vocab = sorted([word for word, count in counter_all.items() if len(word) >= 3 and count >= 10])

print("\nКоличество уникальных слов длиной более 3 и частотой более 10:")
print(len(filtered_vocab))
print(filtered_vocab[:10]) 


def vectorize(tokens, vocab):
    word_counts = Counter(tokens)
    return [word_counts[word] for word in vocab]


vectors = df["message"].apply(lambda tokens: vectorize(tokens, filtered_vocab)).tolist()
vector_df = pd.DataFrame(vectors, columns=filtered_vocab)

print(vector_df.head())
nonzero_rows = vector_df[(vector_df != 0).any(axis=1)]



Приведено все к нижнему регистру, удалены символы не являющеся буквами и пробелами.

0       [have, your, lunch, and, come, quickly, and, o...
1       [join, the, uks, horniest, dogging, service, a...
2       [you, are, chosen, to, receive, a, award, pls,...
3       [alright, tylers, got, a, minor, crisis, and, ...
4       [sad, story, of, a, man, last, week, was, my, ...
                              ...                        
3710    [hello, which, the, site, to, download, songs,...
3711    [my, planning, usually, stops, at, find, hella...
3712              [are, u, awake, is, there, snow, there]
3713                            [sorry, ill, call, later]
3714                       [you, can, never, do, nothing]
Name: message, Length: 3715, dtype: object

Лемматизация

0       [have, your, lunch, and, come, quickly, and, o...
1       [join, the, uk, horniest, dogging, service, an...
2       [you, are, chosen, to, receive, a, award, pls,...
3       [alright, tyler, got, a, minor, cris

Задание Spelling Bee.
Для игры создан словарь английских слов (файл джейсон) с просчитанными очками для каждого слова для ускорения. Случайные слова стали супер словами, за которые будет выдано больше очков.
Сама игра находится в файле sbellingBee.py, а запустить ее можно из main

In [ ]:

import json
import random

def calculate_score(word):
    length = len(word)
    if length < 4:
        return 0
    elif length == 4:
        return 1
    elif 5 <= length <= 7:
        return length
    else:
        return length * 2

def assign_scores(word_dict):
    new_dict = {}
    for word in word_dict.keys():
        score = calculate_score(word)
        if score > 0:
            new_dict[word.lower()] = score
    return new_dict

def add_super_words(word_score_dict, super_multiplier=2, super_ratio=0.02):
    words = list(word_score_dict.keys())
    super_words = random.sample(words, int(len(words) * super_ratio))

    for word in super_words:
        word_score_dict[word] *= super_multiplier
    return word_score_dict

with open("words_dictionary.json", "r") as f:
    original_dict = json.load(f)

scored_words = assign_scores(original_dict)
scored_with_supers = add_super_words(scored_words)

with open("scored_words.json", "w") as f:
    json.dump(scored_with_supers, f, indent=2)